In [1]:
import json
import cv2
import keras_tuner as kt
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

Using TensorFlow backend


---
### Data Reading
---

In [2]:
data = np.load('../data/training_data_arg.npz', allow_pickle=True)

train_set = data['train_set']
test_set = data['test_set']
val_set = data['val_set']

In [3]:
def resize_image(image, target_size):
    # Convert the image to 3 channels if it's not
    if len(image.shape) == 2:
        image = np.stack((image,)*3, axis=-1)
    # Resize the image
    resized_image = cv2.resize(image, target_size)
    return resized_image

In [4]:
def create_dataframe(set):
    df_ = pd.DataFrame([], columns=['x', 'y'])	
    for i in range(len(set)):
        df_ = pd.concat([df_, pd.DataFrame(
            {'reading': [resize_image(set[i]['x'].reshape(24, 32), (32, 32))],
            'posture': set[i]['y']}
        )])
    return df_

In [5]:
df_train = create_dataframe(train_set)
df_test = create_dataframe(test_set)
df_val = create_dataframe(val_set)

print(df_train.shape, df_test.shape, df_val.shape)

(1620, 4) (54, 4) (405, 4)


In [6]:
X_train = df_train['reading']
y_train = df_train['posture']
X_val = df_val['reading']
y_val = df_val['posture']
X_test = df_test['reading']
y_test = df_test['posture']
# One-hot encode the labels
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_val).values
y_test = pd.get_dummies(y_test).values

In [7]:
X_train = np.array([np.array(x) for x in X_train])
# X_train = np.expand_dims(X_train, -1)
X_val = np.array([np.array(x) for x in X_val])
# X_val = np.expand_dims(X_val, -1)
X_test = np.array([np.array(x) for x in X_test])
# X_test = np.expand_dims(X_test, -1)
y_train = np.array([np.array(x) for x in y_train])
y_test = np.array([np.array(x) for x in y_test])
y_val = np.array([np.array(x) for x in y_val])
# Convert the data to tensors
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

---
### Model Definition
---

In [8]:
base_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(32, 32, 3)
)

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

base_model.trainable = False

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    patience=5
)

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

---
### Model Training
---

In [11]:
hist = model.fit(
    X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=100, callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
51/51 [==============================] - 14s 227ms/step - loss: 0.6968 - accuracy: 0.7142 - val_loss: 1.4061 - val_accuracy: 0.5951 - lr: 0.0010
Epoch 2/100
51/51 [==============================] - 11s 220ms/step - loss: 0.3360 - accuracy: 0.8519 - val_loss: 1.4322 - val_accuracy: 0.5383 - lr: 0.0010
Epoch 3/100
51/51 [==============================] - 12s 234ms/step - loss: 0.2409 - accuracy: 0.9031 - val_loss: 1.6517 - val_accuracy: 0.5926 - lr: 0.0010
Epoch 4/100
51/51 [==============================] - 11s 225ms/step - loss: 0.2065 - accuracy: 0.9185 - val_loss: 1.5788 - val_accuracy: 0.6593 - lr: 0.0010
Epoch 5/100
51/51 [==============================] - 11s 225ms/step - loss: 0.1551 - accuracy: 0.9438 - val_loss: 1.7850 - val_accuracy: 0.5778 - lr: 0.0010
Epoch 6/100
51/51 [==============================] - 12s 226ms/step - loss: 0.1341 - accuracy: 0.9475 - val_loss: 1.8905 - val_accuracy: 0.6370 - lr: 0.0010
Epoch 7/100
51/51 [==============================] - 11s 2

In [14]:
px.line(
    hist.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'},
    title='Training and Validation Loss'
)

In [12]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 1s 156ms/step - loss: 1.1296 - accuracy: 0.5556


[1.1296327114105225, 0.5555555820465088]

In [13]:
model.save('vgg16_da.keras')